### Сетап

In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, get_scheduler, default_data_collator
from accelerate import Accelerator
from tqdm.auto import tqdm
import numpy as np
import evaluate
import collections
import ipywidgets as widgets

In [2]:
DATASET_NAME = "kuznetsoffandrey/sberquad"
MODEL_NAME = "distilbert/distilbert-base-multilingual-cased"
MODEL_SAVE_DIR ='distilbert-v2'

### Загрузка датасета и его предобработка

In [3]:
dataset = load_dataset(DATASET_NAME)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [5]:
dataset['validation'][0]

{'id': 60544,
 'title': 'SberChallenge',
 'context': 'Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.',
 'question': 'Где встречаются первые упоминания о строении человеческого тела?',
 'answers': {'text': ['в Древнем Египте'], 'answer_start': [60]}}

In [6]:
dataset['test'][0]

{'id': 18009,
 'title': 'SberChallenge',
 'context': 'Многоклеточный организм — внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых (кроме стволовых, например, клеток камбия у растений) дифференцированы, то есть различаются по строению и выполняемым функциям. Следует отличать многоклеточность и колониальность. У колониальных организмов отсутствуют настоящие дифференцированные клетки, а следовательно, и разделение тела на ткани. Граница между многоклеточностью и колониальностью нечёткая. Например, вольвокс часто относят к колониальным организмам, хотя в его колониях есть чёткое деление клеток на генеративные и соматические. Кроме дифференциации клеток, для многоклеточных характерен и более высокий уровень интеграции, чем для колониальных форм. Многоклеточные животные, возможно, появились на Земле 2,1 миллиарда лет назад, вскоре после кислородной революции .',
 'question': 'У каких организмов отсутствуют настоящие дифференцированные

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

m:\python_projects\ru_qa_system\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
tokenized_example = tokenizer("Какого цвета мои шнурки?", "Мои шнурки серо-буро-малиновые.",
        max_length=50,
        truncation="only_second",
        stride=30,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

tokenized_example

{'input_ids': [[101, 45383, 10990, 42128, 553, 26891, 565, 11752, 46559, 136, 102, 521, 26891, 565, 11752, 46559, 10277, 14315, 118, 18261, 14315, 118, 32034, 105856, 10205, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'offset_mapping': [[(0, 0), (0, 4), (4, 6), (7, 12), (13, 14), (14, 16), (17, 18), (18, 20), (20, 23), (23, 24), (0, 0), (0, 1), (1, 3), (4, 5), (5, 7), (7, 10), (11, 13), (13, 15), (15, 16), (16, 18), (18, 20), (20, 21), (21, 25), (25, 29), (29, 30), (30, 31), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]], 'overflow_to_sample_mapping': [0]}

In [9]:
list(tokenized_example.keys())

['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping']

In [10]:
def preprocess_data(examples, tokenizer, is_test=False, max_length=384, stride=128):
    questions = [q.strip() for q in examples['question']]
    
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    answers = examples["answers"]
    if is_test:
        offset_mapping = inputs["offset_mapping"]
        example_ids = []
    else:
        offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        offset = offset_mapping[i]
        sequence_ids = inputs.sequence_ids(i)

        if is_test:
            example_ids.append(examples["id"][sample_idx])
            inputs["offset_mapping"][i] = [
                o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
            ]

        if len(answer['answer_start'])==0:
            start_positions.append(0)
            end_positions.append(0)
            continue
        
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    if is_test:
        inputs["example_id"] = example_ids
    return inputs

In [11]:
dataset_train = dataset['train'].map(
    preprocess_data,
    batched=True,
    remove_columns=dataset['train'].column_names,
    fn_kwargs = {
        'tokenizer': tokenizer,
    }
)

Map:   0%|          | 0/45328 [00:00<?, ? examples/s]

In [12]:
dataset_train

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 47782
})

In [13]:
dataset_val = dataset['validation'].map(
    preprocess_data,
    batched=True,
    remove_columns=dataset['validation'].column_names,
    fn_kwargs = {
        'tokenizer': tokenizer,
        'is_test': True,
    }
)

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

In [14]:
dataset_val

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
    num_rows: 5316
})

In [15]:
dataset_val[0]

{'input_ids': [101,
  512,
  12265,
  78007,
  37067,
  560,
  91363,
  13139,
  12268,
  555,
  19240,
  102875,
  11905,
  25877,
  16030,
  19069,
  33045,
  136,
  102,
  59186,
  560,
  91363,
  13139,
  12268,
  555,
  19240,
  102875,
  11905,
  25877,
  16030,
  19069,
  33045,
  78007,
  543,
  513,
  110442,
  30072,
  514,
  14122,
  11078,
  10696,
  119,
  511,
  102974,
  30025,
  32479,
  10344,
  554,
  119,
  570,
  119,
  546,
  14122,
  107006,
  11386,
  95739,
  517,
  10241,
  42940,
  86121,
  555,
  75356,
  30847,
  79987,
  549,
  12064,
  44490,
  117,
  543,
  27184,
  64371,
  11075,
  553,
  44666,
  10823,
  117,
  34112,
  10277,
  23479,
  11456,
  117,
  98826,
  83191,
  10297,
  10956,
  16417,
  41127,
  119,
  511,
  16522,
  13292,
  10695,
  10648,
  25987,
  11106,
  48658,
  21124,
  10384,
  12181,
  47397,
  113,
  14627,
  100,
  12988,
  60345,
  119,
  10344,
  554,
  119,
  570,
  119,
  114,
  560,
  91363,
  13139,
  17601,
  10277,
  9

In [16]:
dataset_train.set_format("torch")

dataset_val_formatted = dataset_val.remove_columns(["example_id", "offset_mapping"])
dataset_val_formatted.set_format("torch")

In [17]:
dataloader_train = DataLoader(
    dataset_train,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=16
)

dataloader_val = DataLoader(
    dataset_val_formatted,
    collate_fn=default_data_collator,
    batch_size=16
)

### Определение функций для оценки тренировки модели

In [18]:
def format_predictions(start_logits, end_logits, inputs, examples, n_best=20, max_answer_length=30):
    assert n_best <= len(inputs['offset_mapping'][0]), 'n_best cannot be larger than max_length'
    
    example_to_inputs = collections.defaultdict(list)
    for idx, feature in enumerate(inputs):
        example_to_inputs[str(feature["example_id"])].append(idx)
    
    predicted_answers = []
    for example in tqdm(examples):
        example_id = str(example["id"])
        context = example["context"]
        answers = []
        
        for feature_index in example_to_inputs[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]

            offsets = inputs[feature_index]['offset_mapping']
            start_indices = np.argsort(start_logit)[-1:-n_best-1:-1].tolist()
            end_indices = np.argsort(end_logit)[-1 :-n_best-1: -1].tolist()

            for start_index in start_indices:
                for end_index in end_indices:
                    if (end_index < start_index or end_index - start_index + 1 > max_answer_length):
                        continue
                    if (offsets[start_index] is None)^(offsets[end_index] is None):
                        continue
                    
                    if (offsets[start_index] is None)&(offsets[end_index] is None):
                        answers.append(
                            {
                                "text": '',
                                "logit_score": start_logit[start_index] + end_logit[end_index],
                            }
                        )

                    else:
                        answers.append(
                            {
                                "text": context[offsets[start_index][0] : offsets[end_index][1]],
                                "logit_score": start_logit[start_index] + end_logit[end_index],
                            }
                        )

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x:x['logit_score'])
            predicted_answers.append({'id':example_id, 'prediction_text':best_answer['text']})
        else:
            predicted_answers.append({'id':example_id, 'prediction_text':''})

    return predicted_answers

In [19]:
def compute_metrics(start_logits, end_logits, inputs, examples, n_best = 20, max_answer_length=30):

    metric = evaluate.load('squad_v2')
    predicted_answers = format_predictions(start_logits, end_logits, inputs, examples,
                                           n_best=n_best, max_answer_length=max_answer_length)
    for pred in predicted_answers:
        pred['no_answer_probability'] = 1.0 if pred['prediction_text'] == '' else 0.0

    correct_answers = []
    for example in examples:
        input_id = str(example["id"])
        answers = example["answers"]
        correct_answers.append({
            "id": input_id,
            "answers": {
                "text": answers["text"],
                "answer_start": answers["answer_start"]
            }
        })
    return metric.compute(predictions=predicted_answers, references=correct_answers)

### Тренировка модели

In [20]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
optimizer = AdamW(model.parameters(), lr=3e-5)

num_epochs = 2
num_training_steps = len(dataloader_train)*num_epochs

scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
accelerator = Accelerator(mixed_precision="fp16")

model, optimizer, dataloader_train, dataloader_val = accelerator.prepare(
    model, optimizer, dataloader_train, dataloader_val
)

m:\python_projects\ru_qa_system\venv\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [22]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    # Тренировка модели
    model.train()
    for step, batch in enumerate(dataloader_train):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # Оценка на валидационных данных
    model.eval()
    start_logits = []
    end_logits = []
    for batch in tqdm(dataloader_val, desc='Оценка модели'):
        with torch.no_grad():
            outputs = model(**batch)
        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())
    
    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(dataset_val)]
    end_logits = end_logits[: len(dataset_val)]
    
    metrics = compute_metrics(
        start_logits, end_logits, dataset_val, dataset['validation']
    )
    print(f"Эпоха {epoch+1}:", metrics)
    
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(MODEL_SAVE_DIR,save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(MODEL_SAVE_DIR)

  0%|          | 0/5974 [00:00<?, ?it/s]

Оценка модели:   0%|          | 0/333 [00:00<?, ?it/s]

  0%|          | 0/5036 [00:00<?, ?it/s]

Эпоха 1: {'exact': 53.05798252581414, 'f1': 73.12466310014788, 'total': 5036, 'HasAns_exact': 53.05798252581414, 'HasAns_f1': 73.12466310014788, 'HasAns_total': 5036, 'best_exact': 53.05798252581414, 'best_exact_thresh': 0.0, 'best_f1': 73.12466310014788, 'best_f1_thresh': 0.0}


Оценка модели:   0%|          | 0/333 [00:00<?, ?it/s]

  0%|          | 0/5036 [00:00<?, ?it/s]

Эпоха 2: {'exact': 54.76568705321684, 'f1': 74.41162288203093, 'total': 5036, 'HasAns_exact': 54.76568705321684, 'HasAns_f1': 74.41162288203093, 'HasAns_total': 5036, 'best_exact': 54.76568705321684, 'best_exact_thresh': 0.0, 'best_f1': 74.41162288203093, 'best_f1_thresh': 0.0}


Лучшие метрики были получены на 2ой эпохе:

> Exact match = 54.77 \
> F1-score = 74.41

### Сохранение модели

In [23]:
tokenizer.save_pretrained(MODEL_SAVE_DIR)
model.save_pretrained(MODEL_SAVE_DIR)